<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Public_MySQL_Server_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Goal
In this Notebook, we install a MySQL Database on the Google Colab Platform and then make it visible to in the public internet using NGROK Tunnels

#Create Local MySQL Database

## Install MySQL

In [1]:
!apt update > /dev/null
!apt install -y  mysql-server > /dev/null
!/etc/init.d/mysql restart
!mysql -e 'create database praxisDB'
#!mysql -e 'show databases'





 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


## Pythons Functions for Local SQL Access

In [2]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb

In [13]:
#To run any non-SELECT SQL command
def runCMD (loc, DDL):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
    
    myCursor = DBConn.cursor()
    retcode = myCursor.execute(DDL) 
    print (retcode)
    DBConn.commit()
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (loc, CMD):
    if loc == 'remoteDB':
        DBConn = MySQLdb.connect(hostName,userName,passWord,dbName)
    elif loc == 'localDB':
        DBConn = MySQLdb.connect(db='praxisDB')
    else :
        print('location must be "remoteDB or "localDB"')
    
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

## Create and Load Tables EMP, DEPT

In [4]:
# Set the location to whichever Database you need to access
#
# --------------------------------
#
location = 'localDB'
#location = 'remoteDB'

In [14]:
# Create and load Emp table
# -----------------------------
runCMD(location,"DROP TABLE IF EXISTS Emp;")
# -----------------------------
runCMD(location,"CREATE TABLE IF NOT EXISTS Emp (\
  EmpID char(6)  NOT NULL,\
  LastName varchar(50),\
  FirstName varchar(50),\
  JobDesc varchar(50),\
  JoinDate date NOT NULL, \
  Salary int(11) , \
  Comm float , \
  DeptID char(2) \
) ;")
# -----------------------------
runCMD(location,"INSERT INTO Emp (EmpID, LastName, FirstName, JobDesc, JoinDate, Salary, Comm, DeptID) \
    VALUES \
	('742866', 'Bacchan', 'Amitabh', 'Executive', '2003-03-10', 50000, 0.1, '10'), \
	('349870', 'Mukherjee', 'Rani', 'Manager', '2005-05-04', 25000, 0.06, '40'), \
	('865477', 'Dikshit', 'Madhuri', 'Clerk', '2002-04-04', 10000, 0.02, '20'), \
	('239456', 'Khan', 'Shahrukh', 'Manager', '2004-01-03', 30000, 0.07, '20'), \
	('897889', 'Sehwag', 'Virender', 'Cus_Rep', '2005-01-02', 15000, 0.05, '20'), \
	('123980', 'Dhoni', 'Mahender', 'Clerk', '2004-10-09', 9000, 0.02, '40'), \
	('822134', 'Dravid', 'Rahul', 'Sr Manager', '2000-06-04', 40000, 0.08, '30'), \
	('997445', 'Dalmia', 'Jagmohan', 'Clerk', '2001-07-01', 12000, 0.02, '30'), \
	('989007', 'Ganguly', 'Sourav', 'Cus_Rep', '2002-01-01', 20000, 0.03, '40'), \
	('299034', 'Ganesan', 'Rekha', 'Director', '2002-10-10', 60000, 0.11, '10'), \
	('546223', 'Karthikeyan', 'Narayan', 'Secretary', '2005-12-04', 40000, 0.09, '10'), \
	('223112', 'Mirza', 'Sania', 'Cus_Rep', '2001-11-19', 25000, 0.04, '30');"
    )

0
0
12


In [15]:
# Create and load DEPT table
# --------------------------------
runCMD(location,"DROP TABLE IF EXISTS Dept;")
# --------------------------------
runCMD(location,"CREATE TABLE Dept ( \
  DeptID char(2)  NOT NULL, \
  DeptName varchar(50) , \
  ManagerID char(6) , \
  Location varchar(50) \
 );")
# --------------------------------
runCMD(location,"INSERT INTO Dept (DeptID, DeptName, ManagerID, Location) VALUES \
	('10', 'Corporate', '299034', 'Calcutta'), \
	('20', 'Sales', '239456', 'Calcutta'), \
	('30', 'Accounts', '822134', 'Calcutta'), \
	('40', 'Production', '349870', 'Bombay');")

0
0
4


In [7]:
# Check the data
# -------------------------------
#runSELECT(location,"select * from Emp, Dept where Emp.DeptID = Dept.DeptID")

In [15]:
runSELECT(location,'Select * from Emp')

,EmpID,LastName,FirstName,JobDesc,JoinDate,Salary,Comm,DeptID
0,742866,Bacchan,Amitabh,Executive,2003-03-10,50000,0.10,10
1,349870,Mukherjee,Rani,Manager,2005-05-04,25000,0.06,40
2,865477,Dikshit,Madhuri,Clerk,2002-04-04,10000,0.02,20
3,239456,Khan,Shahrukh,Manager,2004-01-03,30000,0.07,20
4,897889,Sehwag,Virender,Cus_Rep,2005-01-02,15000,0.05,20
5,123980,Dhoni,Mahender,Clerk,2004-10-09,9000,0.02,40
6,822134,Dravid,Rahul,Sr Manager,2000-06-04,40000,0.08,30
7,997445,Dalmia,Jagmohan,Clerk,2001-07-01,12000,0.02,30
8,989007,Ganguly,Sourav,Cus_Rep,2002-01-01,20000,0.03,40
9,299034,Ganesan,Rekha,Director,2002-10-10,60000,0.11,10


#Modifications for Remote Access <br>
https://phoenixnap.com/kb/mysql-remote-connection

## Change to mysqld.cnf file

In [8]:
# Here is the original mysqld.cnf file
#!cat /etc/mysql/mysql.conf.d/mysqld.cnf

In [7]:
# This one line needs to be changed
!sed 's/bind-address		= 127.0.0.1/bind-address		= 0.0.0.0/g' /etc/mysql/mysql.conf.d/mysqld.cnf > mysqld_mod.cnf

In [8]:
!cp /content/mysqld_mod.cnf /etc/mysql/mysql.conf.d/mysqld.cnf
!/etc/init.d/mysql restart
#!mysql -e 'show databases'

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


## Setting up Tunnel
https://danielmiessler.com/study/ngrok/

In [9]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/'My Drive'/Praxis/WebCredentials/ngrokToken.py .
# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM

from ngrokToken import ngrokToken

#for the sake of privacy
#the following credentials need to be stored in a text file called ngrokToken.py
#in the format given below
#in the Colab VM

#otherwise, the values can be directly placed here

#ngrokToken = 'xxxxxxxxxxxxxxx'   # uncomment this line and place your own credentials here
#token is available at https://dashboard.ngrok.com/get-started/setup


#print(ngrokToken)
ngrokTokenCmd = 'ngrok authtoken '+ngrokToken

Mounted at /content/drive


### ngrok wrapper

In [10]:
!pip -q install pyngrok

from pyngrok import ngrok
# you may place the token directly here
#!ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
get_ipython().system_raw(ngrokTokenCmd)

# Open a HTTP tunnel on the default port 80
#public_url = ngrok.connect(port = '4050')
#public_url = ngrok.connect(3306)



     |████████████████████████████████| 745 kB 5.2 MB/s 


In [11]:
tcp_tunnel = ngrok.connect(3306, "tcp")
tcp_tunnel

<NgrokTunnel: "tcp://6.tcp.ngrok.io:10754" -> "localhost:3306">

In [17]:
#!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"tcp-3306-96ba550c-3ce8-4b39-ba20-7f91bb642410","uri":"/api/tunnels/tcp-3306-96ba550c-3ce8-4b39-ba20-7f91bb642410","public_url":"tcp://4.tcp.ngrok.io:12435","proto":"tcp","config":{"addr":"localhost:3306","inspect":false},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"http-3306-926c0cdf-b1bc-40b5-a0f6-f96ce86a2ae9","uri":"/api/tunnels/http-3306-926c0cdf-b1bc-40b5-a0f6-f96ce86a2ae9","public_url":"https://43e9-34-83-184-75.ngrok.io","proto":"https","config":{"addr":"http://localhost:3306","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [15]:
ngrok.disconnect('http://43e9-34-83-184-75.ngrok.io')

## Creating Multiple Public Users

In [12]:
for userID in range(10,12):
    print(userID)
    cDBCmd = "create database db4user"+str(userID)
    cUserCmd = "create user 'rUser"+str(userID)+"'@'localhost' identified by 'pw4rUser"+str(userID)+"';"
    grantCmd = "grant all on db4user"+str(userID)+".* to 'rUser"+str(userID)+"'@'localhost';"
    print(cDBCmd)
    print(cUserCmd)
    print(grantCmd)
    runCMD(location,cDBCmd)
    runCMD(location, cUserCmd)
    runCMD(location, grantCmd)

10
create database db4user10
create user 'rUser10'@'localhost' identified by 'pw4rUser10';
grant all on db4user10.* to 'rUser10'@'localhost';
1
0
0
11
create database db4user11
create user 'rUser11'@'localhost' identified by 'pw4rUser11';
grant all on db4user11.* to 'rUser11'@'localhost';
1
0
0


In [19]:
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| db4user10          |
| db4user11          |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


In [21]:
!mysql -e 'SELECT user FROM mysql.user;'

+------------------+
| user             |
+------------------+
| debian-sys-maint |
| mysql.session    |
| mysql.sys        |
| rUser10          |
| rUser11          |
| root             |
+------------------+


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)